# Julian's Small Example

Julian came up with a small $n = 2$ example (excluding sex data) for testing the theory behind solving the problem. This example can easily be extended and then solved using the AlphaRGS module. This is a quick demonstration of using the module within an interactive notebook environment.

In [1]:
import alphargs

## Problem Variables

The variables for Julian's $n = 2$ problem are as follows:
$$
    \Sigma = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix},\quad\
    \bar{\mu} = \begin{bmatrix} 1 \\ 2 \end{bmatrix},\quad\
    \Omega = \begin{bmatrix} \frac{1}{9} & 0 \\ 0 & 4 \end{bmatrix}.
$$
This can easily be extended to include sex data by reflecting those variable across both sires and dams, filling in the remainder with zeros. 
$$
    \Sigma = \begin{bmatrix}
                1 & 0 & 0 & 0 \\
                0 & 1 & 0 & 0 \\
                0 & 0 & 1 & 0 \\
                0 & 0 & 0 & 1
             \end{bmatrix},\quad\
    \bar{\mu} = \begin{bmatrix} 1 \\ 1 \\ 2 \\ 2  \end{bmatrix},\quad\
    \Omega = \begin{bmatrix}
                \frac{1}{9} & 0 & 0 & 0 \\
                0 & \frac{1}{9} & 0 & 0 \\
                0 & 0 & 4 & 0 \\
                0 & 0 & 0 & 4
             \end{bmatrix},\quad\
    \mathcal{S} = \lbrace 1, 3 \rbrace,\quad\
    \mathcal{D} = \lbrace 2, 4 \rbrace.
$$
The matrix variables are stored in [`A04.txt`](A04.txt), [`EBV04.txt`](EBV04.txt), and [`S04.txt`](S04.txt) respectively, where the first and last are matrices in coordinate format. These are then loaded into Python using `load_problem`.

In [2]:
# key problem variables loaded from standard format txt files
sigma, mubar, omega, n = alphargs.load_problem("A04.txt", "EBV04.txt", "S04.txt")

Since we have constructed the problem to have alternating sex data, we may approach it as we did in [`example.py`](../50/example.py) using range iterates.

In [3]:
sires = range(0, n, 2)
dams = range(1, n, 2)

## Gurobi

AlphaRGS defines functions for solving the standard and robust genetic selection problems using the [gurobipy](https://pypi.org/project/gurobipy/) Python interface to Gurobi. Below these are used to solve the above problem for $\lambda = 0.5, \kappa = 1$: 

In [4]:
lam = 0.5
kap = 1

# computes the standard and robust genetic selection solutions
w_std, obj_std = alphargs.gurobi_standard_genetics(sigma, mubar, sires, dams, lam, n)
w_rbs, z_rbs, obj_rbs = alphargs.gurobi_robust_genetics(sigma, mubar, omega, sires, dams, lam, kap, n)

# print a comparison of the two solutions
alphargs.print_compare_solutions(w_std, w_rbs, obj_std, obj_rbs, z2=z_rbs, name1="w_std", name2="w_rbs")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-26
i  w_std    w_rbs
1  0.00000  0.38200
2  0.00000  0.38200
3  0.50000  0.11800
4  0.50000  0.11800

w_std objective: 1.87500
w_rbs objective: 0.77684 (z = 0.37924)
Maximum change: 0.38200
Average change: 0.38200
Minimum change: 0.38200


Finally, we can use `check_uncertainty_constraint` to explore how close our $z\geq \sqrt{w^{T}\Omega w}$ constraint came to equality.

In [5]:
if not alphargs.check_uncertainty_constraint(z_rbs, w_rbs, omega, debug=True):
    raise ValueError


     z: 0.37923871642022844
w'*Ω*w: 0.3792386953366983
  Diff: 2.1083530143961582e-08
